In [ ]:
import Neuralperk_ProxyUser_MainServer.OldServices.old_files.vastAIManager as vastAIManager
import subprocess
import time
import importlib
importlib.reload(vastAIManager)

<module 'vastAIManager' from '/Users/kushgarg/Desktop/Programs/NeuralPerk/Neuralperk_ProxyUser_MainServer/service_VastAIService/vastAIManager.py'>

In [2]:
vast = vastAIManager.VastAIManager()

# Example filters: 1 GPU, at least 8GB GPU RAM, rentable
filters = ["num_gpus=1", "gpu_ram<=10", "rentable=true", "verified=true"]

cheapest = vast.get_cheapest_offer(filters)
print("Cheapest Offer:", cheapest)


Cheapest Offer: {'ask_contract_id': 17691129, 'avail_vol_ask_id': 20186996, 'avail_vol_dph': 0.0005555555555555556, 'avail_vol_size': 688.0, 'bundle_id': 1208129397, 'bundled_results': None, 'bw_nvlink': 0.0, 'cluster_id': None, 'compute_cap': 860, 'cpu_arch': 'amd64', 'cpu_cores': 64, 'cpu_cores_effective': 16.0, 'cpu_ghz': 3.0, 'cpu_name': 'Xeon® E5-2683 v4 ', 'cpu_ram': 32198, 'credit_discount_max': 0.0, 'cuda_max_good': 12.6, 'direct_port_count': 99, 'discount_rate': 0.0, 'discounted_dph_total': 0.07118518518518518, 'discounted_hourly': 0.0, 'disk_bw': 817.2865941843985, 'disk_name': 'nvme', 'disk_space': 317.0, 'dlperf': 17.025286677703726, 'dlperf_per_dphtotal': 239.1689595723208, 'dph_base': 0.06933333333333333, 'dph_total': 0.07118518518518518, 'dph_total_adj': 0.07281278935185186, 'driver_vers': 560035003, 'driver_version': '560.35.03', 'duration': 4773649.42146492, 'end_date': 1761897186.0, 'expected_reliability': 0.0, 'external': None, 'flops_per_dphtotal': 281.8887658688865

In [9]:
def run_cheapest_instance_test(wait_time=1, filters=None):
    """
    Workflow: 
    - Get cheapest offer
    - Create instance
    - Start instance
    - Wait `wait_time` seconds
    - Stop + Destroy instance
    - Return cost summary
    """
    vast = vastAIManager.VastAIManager()

    # 1. Get cheapest offer
    cheapest = vast.get_cheapest_offer(filters)
    if not cheapest:
        vastAIManager.logging.error("No offers found. Aborting test.")
        return None

    offer_id = cheapest["id"]

    # 2. Create instance
    contract_id = vast.create_instance(offer_id)
    if not contract_id:
        vastAIManager.logging.error("Failed to create instance.")
        return None

    # 3. Start instance (use contract_id)
    vast.start_instance(contract_id)

    # 4. Wait
    time.sleep(wait_time)

    # 5. Stop instance (use contract_id)
    vast.stop_instance(contract_id)

    # 6. Get summary BEFORE destroying instance (use contract_id)
    summary = vast.get_instance_summary(contract_id)
    vastAIManager.logging.info(f"Run summary: {summary}")

    # 7. Destroy instance (this removes it from tracking, use contract_id)
    vast.destroy_instance(contract_id)

    return summary


if __name__ == "__main__":
    filters = ["cuda_vers >= 12.0", "num_gpus=1", "gpu_ram>=8"]  # example filters
    summary = run_cheapest_instance_test(wait_time=1, filters=filters)
    print(summary)

{'ContractID': 25678850, 'OfferID': 23515769, 'Status': 'stopped', 'ComputeCost': 0.0014, 'StorageCost': 0.0, 'TotalCost': 0.0014, 'CreatedAt': 'Sun Sep  7 03:38:09 2025', 'StartedAt': 'Sun Sep  7 03:38:10 2025', 'StoppedAt': 'Sun Sep  7 03:38:13 2025', 'SSHHost': 'ssh3.vast.ai', 'SSHPport': 38850}


In [13]:
def run_docker_instance_test(vast, docker_image=None, wait_time=60, filters=None):
    if not filters:
        filters = ["num_gpus=1", "gpu_ram>=4", "rentable=true", "verified=true"]

    # 1. Find cheapest offer
    cheapest = vast.get_cheapest_offer(filters)
    if not cheapest:
        return {"error": "No offers found"}
    print(f"Selected offer: {cheapest}")

    # 2. Create instance
    contract_id = vast.create_instance_with_lodalasan_image(cheapest["id"])

    if not contract_id:
        return {"error": "Failed to create instance"}
    print(f"Instance created: {contract_id}")
    
    # 3. Start instance
    vast.start_instance(contract_id)
    print("Instance started, waiting for it to be ready...")
    time.sleep(120)

    # 5. Wait for app to produce logs
    print(vast.get_instance_logs(contract_id))
    time.sleep(wait_time)

    vast.stop_instance(contract_id)
    print("Instance stopped")
    summary = vast.get_instance_summary(contract_id)
    print(f"Cost summary: {summary}")

    # 8. Destroy
    vast.destroy_instance(contract_id)
    print("Instance destroyed")

    return {
        "offer": cheapest,
        "cost_summary": summary
    }


In [5]:
# Example 1: Simple Python script in Docker
simple_dockerfile = """
FROM python:3.9-slim
WORKDIR /app
RUN echo 'print("Hello from Docker!")' > hello.py
RUN echo 'print("GPU Info:")' >> hello.py
RUN echo 'import subprocess; subprocess.run(["nvidia-smi"])' >> hello.py
CMD ["python", "hello.py"]
"""

# Example 2: Blender rendering Dockerfile
blender_dockerfile = """
FROM nvidia/cuda:11.8-devel-ubuntu20.04
RUN apt-get update && apt-get install -y \\
    blender \\
    python3 \\
    python3-pip \\
    wget
WORKDIR /app
RUN echo 'import bpy; bpy.ops.render.render(); print("Render complete!")' > render_script.py
CMD ["blender", "--background", "--python", "render_script.py"]
"""

# Example 3: Custom NeuralPerk processing
neuralperk_dockerfile = """
FROM nvidia/cuda:11.8-devel-ubuntu20.04
RUN apt-get update && apt-get install -y \\
    python3 \\
    python3-pip \\
    git
WORKDIR /app
RUN pip3 install torch torchvision torchaudio
RUN echo 'import torch; print(f"PyTorch version: {torch.__version__}"); print(f"CUDA available: {torch.cuda.is_available()}"); print(f"GPU count: {torch.cuda.device_count()}")' > gpu_test.py
CMD ["python3", "gpu_test.py"]
"""

print("Docker examples ready!")

Docker examples ready!


In [14]:
# Test 1: Run simple Python Docker container
print("=== Test 1: Simple Python Docker ===")
filters = ["num_gpus=1", "gpu_ram<=10", "rentable=true", "verified=true"]

# Uncomment to run:
vast = vastAIManager.VastAIManager()
result1 = run_docker_instance_test(
    vast=vast,
    filters=filters
)
print("Result:", result1)

=== Test 1: Simple Python Docker ===
Selected offer: {'ask_contract_id': 18162680, 'avail_vol_ask_id': 20292716, 'avail_vol_dph': 0.0004629629629629629, 'avail_vol_size': 779.0, 'bundle_id': 1208131447, 'bundled_results': None, 'bw_nvlink': 0.0, 'cluster_id': None, 'compute_cap': 860, 'cpu_arch': 'amd64', 'cpu_cores': 64, 'cpu_cores_effective': 16.0, 'cpu_ghz': 3.0, 'cpu_name': 'Xeon® E5-2683 v4 ', 'cpu_ram': 32170, 'credit_discount_max': 0.0, 'cuda_max_good': 12.6, 'direct_port_count': 99, 'discount_rate': 0.0, 'discounted_dph_total': 0.061388888888888896, 'discounted_hourly': 0.0, 'disk_bw': 1848.4, 'disk_name': 'HS-SSD-C2000Pro 2048G', 'disk_space': 386.0, 'dlperf': 16.84780641838247, 'dlperf_per_dphtotal': 274.4439054578139, 'dph_base': 0.060000000000000005, 'dph_total': 0.061388888888888896, 'dph_total_adj': 0.06282118055555556, 'driver_vers': 560035003, 'driver_version': '560.35.03', 'duration': 9974309.81074524, 'end_date': 1767171506.0, 'expected_reliability': 0.0, 'external': 

In [ ]:
# Test 1: Basic VastAI functionality
print("=== Testing VastAI Basic Functionality ===")

try:
    # Test getting offers
    filters = ["num_gpus=1", "gpu_ram>=4", "rentable=true", "verified=true"]
    offers = vast.list_offers(filters)
    print(f"Found {len(offers) if offers else 0} offers")
    
    if offers:
        cheapest = vast.get_cheapest_offer(filters)
        print(f"Cheapest offer: ${cheapest.get('dph_total', 'N/A')}/hour")
        print(f"Offer ID: {cheapest.get('offer_id', 'N/A')}")
        print(f"GPU: {cheapest.get('gpu_name', 'N/A')}")
        print(f"GPU RAM: {cheapest.get('gpu_ram', 'N/A')} GB")
    else:
        print("No offers found with current filters")
        
except Exception as e:
    print(f"Error testing basic functionality: {e}")

print("\n" + "="*50)

In [ ]:
# Test 2: Docker Integration Test
print("=== Testing Docker Integration ===")

try:
    # Run simple Docker test with minimal wait time
    result = run_docker_instance_test(
        dockerfile_content=simple_dockerfile,
        wait_time=10,  # Short wait time for testing
        filters=["num_gpus=1", "gpu_ram>=4", "rentable=true", "verified=true"]
    )
    
    print("Docker Test Results:")
    print(f"  Offer used: {result.get('offer', {}).get('offer_id', 'N/A')}")
    print(f"  Cost per hour: ${result.get('offer', {}).get('dph_total', 'N/A')}")
    
    if 'docker_results' in result:
        docker_res = result['docker_results']
        print(f"  Docker operations: {list(docker_res.keys())}")
        
        if 'run_container' in docker_res:
            container_output = docker_res['run_container']
            print(f"  Container output preview: {container_output[:200] if container_output else 'No output'}")
    
    if 'cost_summary' in result:
        cost = result['cost_summary']
        print(f"  Total cost: ${cost.get('total_cost', 'N/A')}")
        print(f"  Runtime: {cost.get('runtime_seconds', 'N/A')} seconds")
        
except Exception as e:
    print(f"Error in Docker integration test: {e}")

print("\n" + "="*50)

In [ ]:
# Run the basic test first to verify VastAI connection
print("Starting VastAI Integration Tests...")
print("Note: This will create, run, and destroy actual VastAI instances which will incur costs!")
print("Make sure you have VastAI CLI properly configured with your API key.")
print("")

# Uncomment the line below to run basic functionality test
# (This just lists offers and doesn't create instances, so it's safe)
# exec(open('testing.ipynb').read())  # Run the basic test cell above

In [13]:
# Test 2: Run GPU-enabled PyTorch container
print("=== Test 2: GPU PyTorch Docker ===")
gpu_filters = ["num_gpus=1", "gpu_ram>=8", "cuda_vers>=11.0", "rentable=true", "verified=true"]

# Uncomment to run:
# result2 = run_docker_instance_test(
#     dockerfile_content=neuralperk_dockerfile,
#     wait_time=120,
#     filters=gpu_filters
# )
# print("GPU Test Result:", result2)

=== Test 2: GPU PyTorch Docker ===


In [14]:
# Test 3: Run existing Docker image with custom command
print("=== Test 3: Existing Docker Image ===")

# Uncomment to run with existing image:
# result3 = run_docker_instance_test(
#     docker_image="nvidia/cuda:11.8-runtime-ubuntu20.04",
#     docker_command="nvidia-smi",
#     wait_time=30,
#     filters=gpu_filters
# )
# print("Existing Image Result:", result3)

=== Test 3: Existing Docker Image ===


In [18]:
# Quick test: Create instance and test basic command execution
print("=== Quick Command Test ===")
filters = ["num_gpus=1", "gpu_ram>=4", "rentable=true", "verified=true"]

# Find cheapest offer
offers = vast.list_offers(filters)
if offers:
    cheapest = vast.get_cheapest_offer(filters)
    print(f"Selected offer: {cheapest}")
    
    # Create and start instance
    offer_id = vast.create_instance(cheapest["offer_id"], disk=32)
    if offer_id:
        print(f"Instance created: {offer_id}")
        vast.start_instance(offer_id)
        print("Instance started, waiting 30 seconds...")
        time.sleep(30)
        
        # Test simple commands
        print("Testing basic commands:")
        
        commands_to_test = [
            "whoami",
            "pwd",
            "ls -la",
            "which docker",
            "docker --version"
        ]
        
        for cmd in commands_to_test:
            result = vast.execute_command_on_instance(offer_id, cmd)
            print(f"  '{cmd}': {result[:100] if result else 'No output'}")
        
        # Clean up
        vast.stop_instance(offer_id)
        vast.destroy_instance(offer_id)
        print("Instance cleaned up")
    else:
        print("Failed to create instance")
else:
    print("No offers found")

=== Quick Command Test ===
Selected offer: {'offer_id': '25244183', 'gpu': 'RTX_3060', 'price': '0.0481', 'dlperf': 'verified', 'reliability': 'South_Korea,_KR'}
Selected offer: {'offer_id': '25244183', 'gpu': 'RTX_3060', 'price': '0.0481', 'dlperf': 'verified', 'reliability': 'South_Korea,_KR'}
Instance created: 25655337
Instance started, waiting 30 seconds...
Instance created: 25655337
Instance started, waiting 30 seconds...
Testing basic commands:
  'whoami': No output
  'pwd': No output
  'ls -la': No output
  'which docker': No output
  'docker --version': No output
Instance cleaned up
Testing basic commands:
  'whoami': No output
  'pwd': No output
  'ls -la': No output
  'which docker': No output
  'docker --version': No output
Instance cleaned up
